In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,ConstantKernel,Matern
import pickle
from sbi.utils import get_density_thresholder, RestrictedPrior, BoxUniform,process_prior
from torch.distributions import MultivariateNormal


from sbi_ice.utils import plotting_utils,custom_priors,plotting_utils,misc
from sbi_ice.simulators import Layer_Tracing_Sim as lts
import sbi_ice.utils.noise_model as noise_model
data_dir,output_dir = misc.get_data_output_dirs()
work_folder = misc.get_project_root()
color_opts = plotting_utils.setup_plots()


First we set up the geometry object we will use and define which setup we want for the geometry/velocity

In [ ]:
# name = "Ekstrom"
# data_file = Path(data_dir,name,"setup_files","flowtube_full.csv")


name = "Synthetic_long"
data_file = Path(data_dir,name,"setup_files","flowtube_tmb_final.csv")

df = pd.read_csv(data_file)
xmb = df["x_coord"].to_numpy() #x - coordinates of domain
tmb = df["tmb"].to_numpy() #total mass balance
Lx = xmb.max() - xmb.min()

nx_iso             = 500 # Number of points in the x-direction
ny_iso             = 1 # Number of points in the y-direction
dt                 = 0.5 # [yr] timestep for advection scheme


geom = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso) #This will be the main object we interact with





In the following we set up the smb and bmb fields. It is of course possible to just set, e.g. "smb = 0.3" everywhere.
However, here we define a prior distribution over possible smb fields and then sample form that distribution.

In [ ]:
torch.manual_seed(123456) #Set the seed for reproducibility

#We use a perturbed GP prior. First, we have a regular GP prior with mean 0, variance 1, and a specified lengthscale (with a Matern kernel).
#We then perturb the GP samples by multiplying them by a variance factor and adding a constant mean, which are independently sampled from specified distributions.


GP_mean_mu= 0.5 #If GP mean sampled from a Gaussian, this is the mean of that Gaussian
GP_mean_sd= 0.25 #If GP mean sampled from a Gaussian, this is the standard deviation of that Gaussian
# GP_mean_min = 0.0 #If GP mean sampled from a uniform, this is the minimum of that uniform
# GP_mean_max = 1.0 #If GP mean sampled from a uniform, this is the maximum of that uniform
GP_var_min= 0.1 #Minimum variance of GP
GP_var_max= 0.3 #Maximum variance of GP
length_scale= 2.5E+3 #Length scale of GP
nu= 2.5 #Smoothness of GP

ker =  Matern(length_scale=length_scale,nu=nu) #Matern kernel as Gaussian is too smooth
gpr = GaussianProcessRegressor(kernel=ker)
mvn_mean,mvn_cov = gpr.predict(xmb.reshape(-1,1),return_cov=True) #We take the usual GP marginalized covariance on our grid, and make sure it is positive definite by adding a small diagonal matrix
eps = 1e-6
a = np.zeros(shape = mvn_cov.shape)
np.fill_diagonal(a,eps)
mvn_cov += a
mvn_mean = torch.from_numpy(mvn_mean)
mvn_cov = torch.from_numpy(mvn_cov)
custom_prior = custom_priors.AppendedMVN(torch.tensor([GP_mean_mu]),torch.tensor([GP_mean_sd]),torch.tensor([GP_var_min]),
                                            torch.tensor([GP_var_max]),mvn_mean,mvn_cov)

# This is a function from the sbi utils, making it easier to sample from the prior and enforces strict bounds on the prior.
prior, *_ = process_prior(custom_prior,
                            custom_prior_wrapper_kwargs=dict(lower_bound=torch.cat((torch.tensor([GP_mean_mu-3*GP_mean_sd,GP_var_min]),-5.0*torch.ones(mvn_mean.size()))), 
                                                            upper_bound=torch.cat((torch.tensor([GP_mean_mu+3*GP_mean_sd,GP_var_max]),5.0*torch.ones(mvn_mean.size())))))


# prior, *_ = process_prior(custom_prior,
#                             custom_prior_wrapper_kwargs=dict(lower_bound=torch.cat((torch.tensor([GP_mean_min,GP_var_min]),-2.0*torch.ones(mvn_mean.size()))), 
#                                                             upper_bound=torch.cat((torch.tensor([GP_mean_max,GP_var_max]),2.0*torch.ones(mvn_mean.size())))))

#We sample a lot of smbs from this prior for plotting purposes, but only take one to use in the simulation.
samples = custom_prior.sample((1,)).numpy()
bmb_list = []
smb_cnst_mean_array = samples[:,0]
smb_sd_array= samples[:,1]
smb_unperturbed_array = samples[:,2:]
smb_samples = np.expand_dims(smb_cnst_mean_array,1) + np.expand_dims(smb_sd_array,1)*smb_unperturbed_array

smb_cnst_mean = smb_cnst_mean_array[0].copy()
smb_sd = smb_sd_array[0].copy()
smb_unperturbed = smb_unperturbed_array[0].copy()
smb = smb_samples[0].copy()

#Plot the smb samples
fig,ax = plt.subplots(1,1,figsize = (4,2))
ax.plot(xmb,smb_samples[:20].T,color = plotting_utils.color_opts["colors"]["prior"],alpha = 0.3)
print(smb_cnst_mean,smb_sd)
#ax.plot(xmb,smb)
#Finally, set the smb and bmb fields of the geom object.
smb_regrid,bmb_regrid = lts.init_geom_from_fname(geom,data_file,smb=smb)

In [ ]:
print("smb_cnst_mean: ",smb_cnst_mean)
print("smb_sd: ",smb_sd)
print("smb_unperturbed: ",smb_unperturbed)


Now, we define the timestepping of the layer tracing scheme. In general, we want the near surface layers at a higher resolution, so what I tend to do is to simulate in 2 phases. The first is quite coarse in terms of vertical resolution, and the second is finer.

In [ ]:
n_surface_phase1 = 25
n_base_phase1 = 100
time_phase1 = 500
n_surface_phase2 = 2
n_base_phase2 = 20
time_phase2 = 500

sched1 = lts.Scheduele(time_phase1,dt,n_surface_phase1,n_base_phase1)
sched2 = lts.Scheduele(time_phase2,dt,n_surface_phase2,n_base_phase2)
print(sched1.total_iterations)
print(sched2.total_iterations)

In [ ]:
#Now simulate

geom.initialize_layers(sched1,10)
lts.sim(geom,smb_regrid,bmb_regrid,sched1)
#Can save intermediate results here if we want
geom.initialize_layers(sched2,0)
lts.sim(geom,smb_regrid,bmb_regrid,sched2)

In [ ]:
#Plot select layers
plot_indices = [-50,-100,-200,-300]
print(geom.age_iso[plot_indices])
fig,axs = plotting_utils.plot_isochrones_1d(geom.x,geom.bs.flatten(),geom.ss.flatten(),geom.dsum_iso[:,0,plot_indices],geom.age_iso[plot_indices],bmb_regrid,smb_regrid,real_layers=None,trackers=geom.extract_active_trackers())
axs[1].set_ylim(-300,100)

In principle, we are done now. However, when doing inference, it is important to capture the observational noise in our layers. The code below does this.

In [ ]:
import pickle
from sbi_ice.loaders import ShortProfileLoader
from pathlib import Path
from scipy.fft import rfftfreq



# loader = ShortProfileLoader.ShortProfileLoader(Path(data_dir,"Ekstrom","setup_files"),Path(output_dir,"Ekstrom"),"exp2",gt_version="v0")
# PSD_dict = pickle.load(Path(output_dir,"Ekstrom","exp2","PSD_matched_noise.p").open("rb"))
# #print(PSD_dict)
# freq = PSD_dict["freqs"][0]
# PSD_log_mean = PSD_dict["PSD_log_diffs_means"][0]
# PSD_log_var = PSD_dict["PSD_log_diffs_vars"][0]

freq = torch.from_numpy(rfftfreq(geom.x.size,d=geom.dx))
print(freq)
PSD_log_mean = -8.0*(1-torch.exp(-freq/(5e-4)))
PSD_log_var = 0.5*torch.ones_like(freq)
PSD_dict = {"freqs":[freq],"PSD_log_diffs_means":[PSD_log_mean],"PSD_log_diffs_vars":[PSD_log_var]}


#The function that adds noise takes positions of the layers in (x,depth) rather than (x,elevation), so we first convert the layer elevations to depths.
layers = torch.from_numpy(geom.dsum_iso[:,0,:].copy()).T


layers_thickness = geom.dsum_iso[:,0,:].T
heights = layers_thickness + geom.bs.flatten()
layer_depths = geom.ss.flatten() - heights
layer_depths = torch.from_numpy(layer_depths)
layer_depths = torch.flip(layer_depths,dims=(0,))

#Repeat the x array to have the same shape as the layers
layer_xs = torch.stack([torch.Tensor(geom.x) for i in range(layer_depths.shape[0])])

#Add noise to the layers. Here we return the base error (function of x), depth correction error (function of z), picking error (from labelling the isochrones), and total error (function of x and z)
#base_error,depth_corr,picking_error,error = modelling_utils.depth_error(layer_xs,layer_depths)
base_error,depth_corr,picking_error,error = noise_model.depth_error(layer_xs,layer_depths,freq,PSD_log_mean,PSD_log_var)

#We flip the error vertically to match with the layer elevations again
flipped_error = torch.flip(error,dims=(0,))
layers += flipped_error

#Plot the noisy layers
plt.plot(geom.x,geom.ss.flatten(),"k")
plt.plot(geom.x,geom.bs.flatten(),"k")
for j in plot_indices:
    plt.plot(geom.x,geom.bs.flatten() + layers[j,:].numpy(),label='Layer {}'.format(j),color="red")
    plt.plot(geom.x,geom.bs.flatten() + geom.dsum_iso[:,0,j],color="blue")




# for layer in loader.real_layers:
#     plt.plot(geom.x,layer,color="green")
plt.ylim(-100,100)

## Save outputs

In [ ]:
shelf = "Synthetic_long"
output_file = Path(output_dir,shelf,"exp3","real_layers.csv") #Save the real layer elevations
supp_file =  Path(output_dir,shelf,"exp3","real_layers_supp.npy") #Save the indices of the layers we save
mb_file =  Path(output_dir,shelf,"exp3","real_mb.p") #Save the smb and bmb fields
noise_dist_file =  Path(output_dir,shelf,"exp3","PSD_matched_noise.p") #Save the noise distribution

plot_indices = np.array(plot_indices,dtype=int)
df = pd.DataFrame(geom.x,columns = ["x_coord"])
for i in range(len(plot_indices)):
    df["layer {}".format(i+1)] = geom.ss.flatten() - (layers.numpy().T[:,plot_indices[i]]+geom.bs.flatten()) #We save layer elevations above sea level instead of cumulative thicknesses

df.to_csv(output_file)
true_df = pd.read_csv(output_file)
np.save(supp_file,plot_indices)

mb_df = pd.DataFrame(np.vstack((geom.x,smb_regrid.flatten(),bmb_regrid.flatten())).T,columns = ["x_coord","smb","bmb"])
with open(mb_file,"wb") as f:
    pickle.dump(dict(x_coord=xmb,smb_const_mean=smb_cnst_mean,smb_var=smb_sd,smb_unperturbed=smb_unperturbed,bmb_regrid=bmb_regrid.flatten()),f)
with open(mb_file,"rb") as f:
    out = pickle.load(f)

with open(noise_dist_file,"wb") as f:
    pickle.dump(PSD_dict,f)
